In [ ]:
!pip install -U autogluon > /dev/null

from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Parameters

In [ ]:
RAND_VAL=42
num_folds=5 ## Number of folds
n_est=3500 ## Number of estimators

## Read Train and Test Files

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
print(df_train.columns)
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_test = pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')
df_test_ov = df_test.copy()
df_test.head()

In [ ]:
# 1) Thư viện
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns          # chỉ dùng để set style, không bắt buộc

# 2) Đọc dữ liệu --------------------------------------------------------------
# Thay đường dẫn này bằng tệp của bạn (.csv, .parquet… tuỳ ý)
df = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')

# 3) Tạo cột Stays (nếu chưa có) ---------------------------------------------
#   - Quy ước phổ biến: Exited = 1  → khách RỜI ĐI
#                       Exited = 0  → khách Ở LẠI
#   - Khi đó, Stays = 1 – Exited
if 'Stays' not in df.columns:
    df['Stays'] = 1 - df['Exited']

# 4) Đếm số mẫu của mỗi lớp ----------------------------------------------------
# Với hai cột nhị phân 0/1, tổng (sum) chính là số bản ghi có giá trị 1
class_counts = df[['Stays', 'Exited']].sum()

print('Class counts:')
print(class_counts)            # để bạn kiểm tra nhanh trên console

# 5) Vẽ biểu đồ ---------------------------------------------------------------
sns.set_style('whitegrid')     # tuỳ chọn: nền sáng + lưới mảnh
plt.figure(figsize=(6,4))

# Ta giữ đúng hai màu như ảnh mẫu: xanh lá & đỏ
class_counts.plot(kind='bar',
                  color=['#148200', '#d62728'],  # xanh lá, đỏ
                  edgecolor='black')

plt.ylabel('Count')
plt.xticks(rotation=0)
plt.title('Label distribution: Stays vs Exited', pad=12)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math

# Nếu df đã tồn tại thì bỏ bước đọc file
# df = pd.read_csv("path/to/your_dataset.csv")

# ─── THIẾT LẬP DANH SÁCH CỘT MUỐN PLOT ───────────────────────────────────────
cols_to_plot = [
    "CreditScore", "Age", "Tenure",
    "Balance", "NumOfProducts", "HasCrCard",
    "IsActiveMember", "EstimatedSalary", "Exited"
]

# Kiểm tra nhanh xem thiếu cột nào không
missing = [c for c in cols_to_plot if c not in df.columns]
if missing:
    raise KeyError(f"Các cột sau chưa có trong DataFrame: {missing}")

# ─── VẼ LƯỚI HISTOGRAM ───────────────────────────────────────────────────────
n_cols = 3                      # số cột trong grid
n_rows = math.ceil(len(cols_to_plot) / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*4, n_rows*3))
axes = axes.flatten()           # chuyển về 1-D để index thuận tiện

for idx, col in enumerate(cols_to_plot):
    ax = axes[idx]
    # Continuous hay categorical đều dùng .hist() được.
    # Thông số bins=auto để matplotlib tự tính hợp lý
    df[col].hist(ax=ax, bins="auto")
    ax.set_title(col, fontweight="bold")
    ax.set_xlabel("")           # bớt rối
    ax.set_ylabel("")           # bớt rối

# Nếu grid thừa ô (khi tổng cột < n_rows*n_cols) ⇒ ẩn các ô trống
for j in range(len(cols_to_plot), len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Variables distribution histogram grid", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------
#  Đếm & vẽ phân bố giới tính (Gender distribution)
# -----------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns               # dùng để set style (tùy chọn)

# 1) Đếm số lượng từng giới tính
gender_counts = df['Gender'].value_counts()      # Hoặc df['Gender'].value_counts(dropna=True)

print('Gender counts:')
print(gender_counts)               # Kiểm tra nhanh trên console

# 2) Vẽ biểu đồ
sns.set_style('whitegrid')
plt.figure(figsize=(6,4))

# Màu giống hình: đỏ cho Male, xanh dương cho Female
colors = ['#d62728', '#1f3d94']     # red, blue
gender_counts.loc[['Male','Female']].plot(kind='bar',
                                          color=colors,
                                          edgecolor='black')

plt.ylabel('Count')
plt.xticks(rotation=0)
plt.title('Gender distribution', pad=12)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math

# Nếu df đã tồn tại thì bỏ bước đọc file
# df = pd.read_csv("path/to/your_dataset.csv")

# ─── THIẾT LẬP DANH SÁCH CỘT MUỐN PLOT ───────────────────────────────────────
cols_to_plot = [
    "CreditScore", "Age", "Tenure",
    "Balance", "NumOfProducts", "HasCrCard",
    "IsActiveMember", "EstimatedSalary", "Exited"
]

# Kiểm tra nhanh xem thiếu cột nào không
missing = [c for c in cols_to_plot if c not in df.columns]
if missing:
    raise KeyError(f"Các cột sau chưa có trong DataFrame: {missing}")

# ─── VẼ LƯỚI HISTOGRAM ───────────────────────────────────────────────────────
n_cols = 3                      # số cột trong grid
n_rows = math.ceil(len(cols_to_plot) / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*4, n_rows*3))
axes = axes.flatten()           # chuyển về 1-D để index thuận tiện

for idx, col in enumerate(cols_to_plot):
    ax = axes[idx]
    # Continuous hay categorical đều dùng .hist() được.
    # Thông số bins=auto để matplotlib tự tính hợp lý
    df[col].hist(ax=ax, bins="auto")
    ax.set_title(col, fontweight="bold")
    ax.set_xlabel("")           # bớt rối
    ax.set_ylabel("")           # bớt rối

# Nếu grid thừa ô (khi tổng cột < n_rows*n_cols) ⇒ ẩn các ô trống
for j in range(len(cols_to_plot), len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Variables distribution histogram grid", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------
#  Fig. 4 – Churn distribution by gender: Count & Percentage
# -----------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1) Bảo đảm có cột Stays
if 'Stays' not in df.columns:
    df['Stays'] = 1 - df['Exited']          # Exited=1 → rời; Stays=1 → ở lại

# 2) Tính tổng Stays/Exited theo giới tính  ------------------
agg_counts = df.groupby('Gender')[['Stays', 'Exited']].sum()

# 3) Chuyển sang phần trăm trong từng giới tính --------------
agg_pct = agg_counts.div(agg_counts.sum(axis=1), axis=0) * 100

# 4) Vẽ hai biểu đồ song song -------------------------------
sns.set_style('whitegrid')
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

colors = ['#1f77b4', '#ffd000']   # xanh dương, vàng (gần giống minh hoạ)

# --- (a) Biểu đồ COUNT
agg_counts.plot(kind='bar',
                ax=axes[0],
                color=colors,
                edgecolor='black')
axes[0].set_title('Churn Risk per Gender (Count)')
axes[0].set_ylabel('Count')
axes[0].set_xlabel('Gender')
axes[0].legend(['Stays', 'Exits'], title='')

# --- (b) Biểu đồ PERCENTAGE
agg_pct.plot(kind='bar',
             ax=axes[1],
             color=colors,
             edgecolor='black')
axes[1].set_title('Churn Risk per Gender (Percentage)')
axes[1].set_ylabel('Percentage')
axes[1].set_xlabel('Gender')
axes[1].legend(['Stays', 'Exits'], title='')

plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------
#  Fig. 5 – Geographical distribution of customers
# -----------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# 1) Đếm số khách theo Geography
geo_counts = (
    df['Geography']
    .value_counts()                       # đếm tất cả địa lý có trong cột
    .loc[['France', 'Germany', 'Spain']] # giữ đúng thứ tự mong muốn
)

print('Customer counts by geography:')
print(geo_counts)

# 2) Vẽ biểu đồ
sns.set_style('whitegrid')
plt.figure(figsize=(7,4))

# Ba màu giống minh hoạ: tím → Pháp, cam → Đức, xanh ngọc → Tây Ban Nha
colors = ['#7b007b', '#f5a623', '#007f7f']   # purple, orange, teal

geo_counts.plot(kind='bar',
                color=colors,
                edgecolor='black')

plt.ylabel('Count')
plt.xlabel('Geography')
plt.title('Geographical distribution of customers', pad=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------
#  Fig. 6 – Churn distribution by geography: Count & Percentage
# -----------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# 1) Đảm bảo có cột Stays
if 'Stays' not in df.columns:
    df['Stays'] = 1 - df['Exited']

# 2) Tính tổng Stays/Exited theo Geography
geo_counts = df.groupby('Geography')[['Stays', 'Exited']].sum()

# 3) Tính phần trăm trong từng Geography
geo_pct = geo_counts.div(geo_counts.sum(axis=1), axis=0) * 100

# 4) Vẽ hai biểu đồ song song
sns.set_style('whitegrid')
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Chọn màu: xanh lá cho Stays, tím cho Exits
colors = ['#148200', '#7b007b']

# (a) Count
geo_counts.plot(kind='bar',
                ax=axes[0],
                color=colors,
                edgecolor='black')
axes[0].set_title('Churn Risk per Geography (Count)')
axes[0].set_ylabel('Count')
axes[0].set_xlabel('Geography')
axes[0].legend(['Stays', 'Exits'], title='')

# (b) Percentage
geo_pct.plot(kind='bar',
             ax=axes[1],
             color=colors,
             edgecolor='black')
axes[1].set_title('Churn Risk per Geography (Percentage)')
axes[1].set_ylabel('Percentage')
axes[1].set_xlabel('Geography')
axes[1].legend(['Stays', 'Exits'], title='')

plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------
#  Fig. 7 – Correlation score heatmap
# -----------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# 1) Tính ma trận tương quan
corr = df[['CreditScore', 'Age', 'Tenure', 'Balance',
           'NumOfProducts', 'HasCrCard',
           'IsActiveMember', 'EstimatedSalary', 'Exited']].corr()

# 2) Vẽ heatmap
plt.figure(figsize=(8, 6))
sns.set_style('white')   # nền trắng để heatmap nổi bật

sns.heatmap(
    corr,
    annot=True,           # hiển thị hệ số tương quan
    fmt='.2f',            # 2 chữ số thập phân
    cmap='RdBu',          # diverging: đỏ + (positive), xanh dương (negative)
    vmin=-1, vmax=1,      # chuẩn hoá về [-1, 1]
    linewidths=0.5,       # kẻ lưới giữa các ô
    cbar_kws={'label': 'Correlation'}
)

plt.title('Correlation Heatmap', pad=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Nếu df đã có sẵn trong RAM thì bỏ bước đọc file
# df = pd.read_csv("path/to/your_dataset.csv") 

# ─── THIẾT LẬP CÁC CỘT CẦN VẼ ────────────────────────────────────────────────
target      = "Exited"                # 0 = Stays, 1 = Exits
violin_cols = ["Age", "Balance", "NumOfProducts", "IsActiveMember"]

# Kiểm tra thiếu cột nào không
missing = [c for c in violin_cols + [target] if c not in df.columns]
if missing:
    raise KeyError(f"Các cột sau chưa có trong DataFrame: {missing}")

# ─── VẼ LƯỚI VIOLIN PLOT ─────────────────────────────────────────────────────
sns.set_style("whitegrid")            # nền sáng giống hình minh hoạ
palette = {0: "forestgreen", 1: "purple"}   # màu xanh & tím như hình

fig, axes = plt.subplots(2, 2, figsize=(11, 8))
axes = axes.flatten()

for i, col in enumerate(violin_cols):
    ax = axes[i]
    # split=False vì ta không vẽ hai nửa chung; mỗi nhóm là một violin riêng
    sns.violinplot(
        data   = df,
        x      = target,
        y      = col,
        palette= palette,
        bw     = 0.2,       # độ mượt; giảm nếu muốn “miền” gọn hơn
        inner  = "box",     # vẽ thêm boxplot bên trong
        ax     = ax
    )
    ax.set_xticklabels(["Stays", "Exits"])
    ax.set_xlabel('Churn Risk ("Exited")')
    ax.set_ylabel(col)
    title = f"Churn vs. {col}" if col != "IsActiveMember" else "Churn vs. Active Membership"
    ax.set_title(title, fontweight="bold")

# Ẩn các ô thừa nếu có (trong trường hợp < 4 biến)
for j in range(len(violin_cols), len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Churn Risk vs. Different Attributes", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()
